In [4]:
"""──────────────────────────────────────────────────────────────────────────┐
│ Loading necessary libraries to build and train model                       │
└──────────────────────────────────────────────────────────────────────────"""
import os,sys,gc
import numpy as np
import pickle,glob
import torch
import proplot as plot
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/')
from tools import derive_var,read_and_proc
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
sys.path.insert(2, '../')
import read_stuff as read
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_3570285/1556194838.py:19: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


In [2]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'
enter = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'

maria_u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
divider = np.asarray([maria_u[0][12:].shape[0],maria_u[1][25:].shape[0],maria_u[2][49:].shape[0],maria_u[3][85:].shape[0],maria_u[4][25:].shape[0]]).cumsum()

  0%|          | 0/5 [00:00<?, ?it/s]

In [83]:
def forward_diff(arrayin=None,delta=None,axis=None,LT=1):
    result = []
    if axis==0:
        for i in range(0,arrayin.shape[axis]-LT):
            temp = (arrayin[i+LT,:]-arrayin[i,:])/(LT*delta)
            result.append(temp)
        return np.asarray(result)
    
def get_time_diff_terms_intermediate(inputvar=None,LT=None,wantvar=None,case=None):
    def _get_time_diff(array=None,timedelta=60*60,LT=None):
        if case=='Maria':
            sizesets = [39,360,int(array[0].shape[1]/39/360)]
        elif case=='Haiyan':
            sizesets = [10,360,int(array[0].shape[1]/10/360)]
        store = []
        for exp in array:
            a = forward_diff(np.nanmean(exp.reshape(exp.shape[0],sizesets[0],sizesets[1],sizesets[2]),axis=2).reshape(exp.shape[0],-1),timedelta,0,LT)
            if a.shape[0]>0:
                azero = np.zeros((LT,exp.shape[-1]))
                store.append(np.asarray(a))
                #store.append(np.concatenate((a,azero),axis=0))
            else:
                continue
                #store.append(np.zeros((exp.shape[0],exp.shape[-1])))
        return store
        
    storedict = {}
    for wantvarZ,wantvarN in enumerate(wantvar):
        storedict[wantvarN] = _get_time_diff(array=inputvar[wantvarZ],LT=LT)
    return storedict
    
def windrates_real_maria(path=None,suffix=None,uvwheatpath=None,LT=None,case='Maria',expstart=[12, 25, 49, 85, 25]):
    u = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['u'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
    v = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['v'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
    w = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['w'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
    theta = [read_and_proc.depickle(path+'TCGphy/2020_TC_CRF/dev/freddy0218/pca/output/uvwheat/preproc2/'+str(lime)+suffix)['theta'] for lime in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf'])]
    
    def innerloop(testindex=None,category=None,u=u,v=v,w=w,theta=theta):
        if category=='train':
            ut = [u[i] for i in range(len(u)) if i not in testindex]
            vt = [v[i] for i in range(len(v)) if i not in testindex]
            wt = [w[i] for i in range(len(w)) if i not in testindex]
            thetat = [theta[i] for i in range(len(theta)) if i not in testindex]
            assert len(ut)==4, 'wrong train-valid-test separation!'
            dtermsT = get_time_diff_terms_intermediate(inputvar=[ut,vt,wt,thetat],LT=LT,wantvar=['u','v','w','theta'],case=case)
        elif category=='valid':
            ut = [u[index] for index in testindex]
            vt = [v[index] for index in testindex]
            wt = [w[index] for index in testindex]
            thetat = [theta[index] for index in testindex]            
            del u,v,w,theta
            gc.collect()
            dtermsT = get_time_diff_terms_intermediate(inputvar=[ut,vt,wt,thetat],LT=LT,wantvar=['u','v','w','theta'],case=case)
        
        tempdict = {}
        for obj in ['u','v','w','theta']:
            tempdict[obj] = dtermsT[obj]
        del dtermsT
        gc.collect()
        return tempdict
    
    storevalid,storetrain = [],[]
    for i in [1,2,3,4]:
        storevalid.append(innerloop([i],'valid'))
        storetrain.append(innerloop([i],'train'))
        
    storetrainout = []
    for indi,obji in enumerate([1,2,3,4]):
        mylist = [0,1,2,3,4]
        mylist.remove(obji)
        dudt = (np.concatenate([storetrain[indi]['u'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0))
        dvdt = (np.concatenate([storetrain[indi]['v'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0))
        dwdt = (np.concatenate([storetrain[indi]['w'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0))
        dthetadt = (np.concatenate([storetrain[indi]['theta'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0))
        storetrainout.append({'du':dudt,'dv':dvdt,'dw':dwdt,'dth':dthetadt})
    del dudt,dvdt,dwdt,dthetadt
    gc.collect()
    
    storevalidout = []
    for indi,obji in enumerate([1,2,3,4]):
        mylist = [obji]
        dudt = np.concatenate([storevalid[indi]['u'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0)
        dvdt = np.concatenate([storevalid[indi]['v'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0)
        dwdt = np.concatenate([storevalid[indi]['w'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0)
        dthetadt = np.concatenate([storevalid[indi]['theta'][indj][expstart[int(objj)]:] for indj,objj in enumerate(mylist)],axis=0)
        storevalidout.append({'du':dudt,'dv':dvdt,'dw':dwdt,'dth':dthetadt})    
    return {'train':storetrainout,'valid':storevalidout}

In [90]:
folder = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/full/'#intermediate/'
Xtrain,Xvalid,ytrain,yvalid = [],[],[],[]
for expname in [1,2,3,4]:
    obj = [read_and_proc.depickle(objs) for objs in sorted(glob.glob(folder+'inputoutput/*test'+str(expname)))]
    Xtrain.append(obj[0])
    Xvalid.append(obj[1])
    ytrain.append(obj[2])
    yvalid.append(obj[3])

TYPE = '3D'
if TYPE=='3D':
    folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/'
    pcastore = read_and_proc.depickle(folderpath+'PCA/PCAdict3D')
elif TYPE=='2D':
    folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/maria/processed/intermediate/'
    pcastore = read_and_proc.depickle(folderpath+'PCA/PCAdict')

In [91]:
wcomps = [54,26,75]
upcs,vpcs,thpcs = pcastore['u'].components_[:wcomps[0]],pcastore['v'].components_[:wcomps[1]],pcastore['theta'].components_[:wcomps[2]]
#[54,26,50,5,75,12,12,10,10]

In [84]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'
yTRUTH= windrates_real_maria(path=path,suffix=suffix,uvwheatpath=None,LT=24,case='Maria')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [165]:
class OptimMLR_all_3D_simple_lwsw(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwsw, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        #self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(2,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([0,54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        bestlw = self.dense1(X_lw)
        bestsw = self.dense2(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [166]:
class OptimMLR_all_3D_simple_lwswu(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwswu, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [54,10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(3,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        bestu = self.dense1(X_u)
        bestlw = self.dense2(X_lw)
        bestsw = self.dense3(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestu,bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [167]:
class OptimMLR_all_3D_simple_lwswu(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwswu, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [54,10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(3,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([0,54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        bestu = self.dense1(X_u)
        bestlw = self.dense2(X_lw)
        bestsw = self.dense3(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestu,bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [168]:
class OptimMLR_all_3D_simple_lwswv(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwswv, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [26,10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(3,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([0,54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        bestv = self.dense1(X_v)
        bestlw = self.dense2(X_lw)
        bestsw = self.dense3(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestv,bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [169]:
class OptimMLR_all_3D_simple_lwsww(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwsww, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [50,10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(3,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([0,54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        bestw = self.dense1(X_w)
        bestlw = self.dense2(X_lw)
        bestsw = self.dense3(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestw,bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [170]:
class OptimMLR_all_3D_simple_lwswth(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwswth, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [75,10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(3,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        bestth = self.dense1(X_th)
        bestlw = self.dense2(X_lw)
        bestsw = self.dense3(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((bestth,bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [171]:
class OptimMLR_all_3D_simple_lwswhdia(torch.nn.Module):
    def __init__(self):
        #super(OptimMLR_all_2D, self).__init__()
        super(OptimMLR_all_3D_simple_lwswhdia, self).__init__()
        ############################################################
        # Input channels
        ############################################################
        brchsize = [12,10,10]#[54,26,50,75,12,10,10]
        self.dense1 = torch.nn.Linear(brchsize[0], 1)
        self.dense2 = torch.nn.Linear(brchsize[1], 1)
        self.dense3 = torch.nn.Linear(brchsize[2], 1)
        #self.dense4 = torch.nn.Linear(brchsize[3], 1)
        #self.dense5 = torch.nn.Linear(brchsize[4], 1)
        #self.dense6 = torch.nn.Linear(brchsize[5], 1)
        #self.dense7 = torch.nn.Linear(brchsize[6], 1)
        ############################################################
        # Final Dense Layer
        ############################################################
        self.denseout = torch.nn.Linear(3,205)#106)
        
    def forward(self,X):
        brchindex = list(np.asarray([54,26,50,75,12,10,10]).cumsum())
        X_u, X_v, X_w, X_th = X[:,brchindex[0]:brchindex[1]],X[:,brchindex[1]:brchindex[2]],X[:,brchindex[2]:brchindex[3]],X[:,brchindex[3]:brchindex[4]]
        X_hdia, X_lw, X_sw = X[:,brchindex[4]:brchindex[5]],X[:,brchindex[5]:brchindex[6]],X[:,brchindex[6]:brchindex[7]]
        ############################################################
        # Optimal PC layer
        ############################################################
        besthdia = self.dense1(X_hdia)
        bestlw = self.dense2(X_lw)
        bestsw = self.dense3(X_sw)
        ############################################################
        # Concat
        ############################################################
        bestPC = torch.cat((besthdia,bestlw,bestsw),1)
        ############################################################
        # Prediction layer
        ############################################################
        outpred = self.denseout(bestPC)
        return outpred

In [96]:
class R2Loss(torch.nn.Module):
    
    def forward(self, y_pred, y):
        var_y = torch.var(y, unbiased=False)
        return -(1.0 - torch.nn.functional.mse_loss(y_pred, y, reduction="mean") / var_y)
    
class MSE_physicalLoss(torch.nn.Module):
    def __init__(self, pcs=None, wantcomps=None, variances=None):
        super(MSE_physicalLoss, self).__init__()
        self.pcs = pcs
        self.wantcomps = wantcomps
        self.variances = variances
        
    def forward(self, y_pred, y):
        def myscore(y_pred, y, var_y):
            return torch.mean(torch.square(y - y_pred).sum()).div(var_y)
            #var_y = torch.var(y, unbiased=False)
            #return (torch.nn.functional.mse_loss(y_pred, y, reduction="mean") / var_y) #mse/var
        """
        1. Here we slice the feature axis to separate samples into u/v/w/theta components
        """
        y_pred_u, y_u = y_pred[:,0:self.wantcomps[0]], y[:,0:self.wantcomps[0]]
        y_pred_v, y_v = y_pred[:,self.wantcomps[0]:self.wantcomps[0]+self.wantcomps[1]], y[:,self.wantcomps[0]:self.wantcomps[0]+self.wantcomps[1]]
        y_pred_w, y_w = y_pred[:,self.wantcomps[0]+self.wantcomps[1]:self.wantcomps[0]+self.wantcomps[1]+self.wantcomps[2]], \
        y[:,self.wantcomps[0]+self.wantcomps[1]:self.wantcomps[0]+self.wantcomps[1]+self.wantcomps[2]]
        y_pred_th, y_th = y_pred[:,self.wantcomps[0]+self.wantcomps[1]+self.wantcomps[2]:self.wantcomps[0]+self.wantcomps[1]+self.wantcomps[2]+self.wantcomps[3]], \
        y[:,self.wantcomps[0]+self.wantcomps[1]+self.wantcomps[2]:self.wantcomps[0]+self.wantcomps[1]+self.wantcomps[2]+self.wantcomps[3]]
        """
        2. Here we convert PCs into physical space
        """
        uphys_pred, uphys_real = torch.matmul(y_pred_u, self.pcs[0]),torch.matmul(y_u, self.pcs[0])
        vphys_pred, vphys_real = torch.matmul(y_pred_v, self.pcs[1]),torch.matmul(y_v, self.pcs[1])
        wphys_pred, wphys_real = torch.matmul(y_pred_w, self.pcs[2]),torch.matmul(y_w, self.pcs[2])
        thphys_pred, thphys_real = torch.matmul(y_pred_th, self.pcs[3]),torch.matmul(y_th, self.pcs[3])
        
        u_mse, v_mse, w_mse, th_mse = myscore(uphys_pred,uphys_real,self.variances[0]),myscore(vphys_pred,vphys_real,self.variances[1]),\
        myscore(wphys_pred,wphys_real,self.variances[2]),myscore(thphys_pred,thphys_real,self.variances[3])
        
        msesum = u_mse+v_mse+w_mse+th_mse
        return msesum/4#, u_mse,v_mse,w_mse,th_mse
    
def eval_model(model, dataloader, loss_func, metric_func):
    with torch.no_grad():
        loss = 0
        metric = 0
        
        global_sum = 0
        label_size = 0
        for feature, labels in dataloader:
            global_sum += labels.sum()
            label_size += len(labels)
            
        global_mean = global_sum / label_size
        for features, labels in dataloader:
            pred = model(features)
            batch_loss = loss_func(pred, labels)
            batch_metric = metric_func(pred, labels, global_mean)
            
            loss+=batch_loss.item()
            metric+=batch_metric.item()
            
        num_batches = len(dataloader)
        
        loss = loss/num_batches
        metric = metric/num_batches
        return (loss, metric)
    
# Customzied evaluation metric NSE for validation set and test set # 
def calc_nse(sim: torch.FloatTensor, obs: torch.FloatTensor, global_obs_mean: torch.FloatTensor) -> float:
    """Calculate the Nash-Sutcliff-Efficiency coefficient.

    :param obs: Array containing the observations
    :param sim: Array containing the simulations
    :param global_obs_mean: mean of the whole observation series
    :return: NSE value.
    """
    numerator = torch.square(sim - obs).sum()
    #denominator = torch.square(obs - global_obs_mean).sum()
    #nse_val = 1 - numerator / denominator

    return numerator

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            #self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
        return self.early_stop

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [97]:
def train_model(optimizer=None,scheduler=None,numepochs=None,early_stopper=None,variance_store=None,lossfunc=None,regularization='None',l1_lambda=0.01):
    # Custom loss: MSE_physicalLoss(eigenvectors,wcomps,variance_store)
    schedulerCY,schedulerLS = scheduler[1],scheduler[0]
    train_losses,trainu_losses,trainv_losses,trainw_losses,trainth_losses = [],[],[],[],[]
    val_losses = []
    val_NSEs = []
    for epoch in tqdm(range(int(num_epochs))):
        """
        Initialize loss
        """
        train_loss = 0
        """
        Operate per batch
        """
        for features, labels in train_loader:
            optimizer.zero_grad()
            
            prediction = model(features)
            batch_loss = lossfunc(prediction, labels)#loss_func(prediction, labels)
            
            if regularization=='L1':
                #L1 regularization-------------------------------
                l1_norm = sum(abs(p) for p in model.parameters())
                #-------------------------------------------------
                batch_loss = batch_loss + l1_lambda * l1_norm
                batch_loss.backward()
            elif regularization=='None':
                batch_loss.backward()                
            
            optimizer.step()
            schedulerCY.step()
            
            train_loss += batch_loss.item()
            
            
        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)
        val_loss, val_NSE = eval_model(model,
                                       val_loader,
                                       lossfunc,
                                       calc_nse)
        schedulerLS.step(val_loss)
        
        ##################################################################
        # Early Stopping (valid / train)
        ##################################################################
        counter = 0
        if len(val_losses)>=1:
            best_score = val_losses[-1]
            if val_loss > best_score:
                counter += 1
                val_NSEs.append(val_NSE)
                val_losses.append(val_loss)
                if counter >= 10:
                    break
            else:
                val_NSEs.append(val_NSE)
                val_losses.append(val_loss)
        else:
            val_NSEs.append(val_NSE)
            val_losses.append(val_loss)
        
        if early_stopper.__call__(val_loss, model):
            break
        #if early_stopper.early_stop(val_loss):             
        #    break
            ##################################################################
        #val_NSEs.append(val_NSE)
        #val_losses.append(val_loss)
        
        if epoch % 500 == 0:
            print((np.log10(train_loss),np.log10(val_loss)))
            
    #return model, {'train':train_losses,'utrain':trainu_losses,'vtrain':trainv_losses,'wtrain':trainw_losses,'thtrain':trainth_losses,'val':val_losses} 
    return model, {'train':train_losses,'val':val_losses} 

In [175]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/TCG_Rad_keras/store/'
folderpath2='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/haiyan/processed/new3D/'
for exp in ['No','u','v','w','th','hdia']:
    for splitnum in range(4):
        X_totrain,y_totrain = read.train_optimizedMLR(folderpath,folderpath2).delete_padding(Xtrain[splitnum],ytrain[splitnum])#yall_orig[splitnum][23][0])
        X_tovalid,y_tovalid = read.train_optimizedMLR(folderpath,folderpath2).delete_padding(Xvalid[splitnum],yvalid[splitnum])#yall_orig[splitnum][23][1])
        #X_totest,y_totest = read.train_optimizedMLR(None,None).delete_padding(Xtest[splitnum]['lwswdtthuvw'],yall[splitnum][23][2])#yall_orig[splitnum][23][2])
        calc_device = 'cpu'
        ###################################################################################
        # Convert numpy arrays into tensors
        ###################################################################################
        train_Xtensor = torch.FloatTensor(X_totrain).to(calc_device)
        train_ytensor = torch.FloatTensor(y_totrain).to(calc_device)
        val_Xtensor = torch.FloatTensor(X_tovalid).to(calc_device)
        val_ytensor = torch.FloatTensor(y_tovalid).to(calc_device)
        train_data = torch.utils.data.TensorDataset(train_Xtensor, train_ytensor)
        val_data = torch.utils.data.TensorDataset(val_Xtensor, val_ytensor)
        batch_size = 9
        num_workers = 2
        train_loader = torch.utils.data.DataLoader(
            dataset=train_data,
            batch_size=batch_size,
            shuffle=True
        )
        val_loader = torch.utils.data.DataLoader(
            dataset=val_data,
            batch_size=batch_size,
            shuffle=False
        )
        times = ['exp1a','exp1b','exp1c','exp1d','exp1e']
        #times = ['exp2a','exp2b','exp2c']#,'exp1d','exp1e']
        for i in range(len(times)):
            models,losses = [],[]
            #model = OptimMLR_all_2D()
            if exp=='No':
                model = OptimMLR_all_3D_simple_lwsw()
            elif exp=='u':
                model = OptimMLR_all_3D_simple_lwswu()
            elif exp=='v':
                model = OptimMLR_all_3D_simple_lwswv()
            elif exp=='w':
                model = OptimMLR_all_3D_simple_lwsww()
            elif exp=='th':
                model = OptimMLR_all_3D_simple_lwswth()
            elif exp=='hdia':
                model = OptimMLR_all_3D_simple_lwswhdia()
            optimizers = [torch.optim.Adam(model.parameters(), lr=1e-6)]#, optim.AdaBound(model.parameters(),lr=1e-7)] 1e-6
            loss = torch.nn.MSELoss()
            for optimizer in optimizers:
                scheduler2 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-9, max_lr=2e-5,cycle_momentum=False)
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',min_lr=1e-18)
                num_epochs = 1000*18#26
                early_stopper = EarlyStopping(patience=50, verbose=False, delta=1e-7, path='checkpoint.pt', trace_func=print)#EarlyStopper(patience=8, min_delta=1e-3)
                #variance_store = [varu,varv,varw,varth]
                #variance_store = [varu,varv,varth]
                model,loss = train_model(optimizer=optimizer,scheduler=[scheduler,scheduler2],numepochs=num_epochs,early_stopper=early_stopper,variance_store=None,\
                                         lossfunc=loss)
                models.append(model)
                losses.append(loss)
                if exp=='No':
                    torch.save(models, '../tmp/maria/3d/lwsw/0/models'+str(splitnum)+'_LWSW3d_1115_'+str(times[i])+'.pt')
                    read_and_proc.save_to_pickle('../tmp/maria/3d/lwsw/0/losses'+str(splitnum)+'_LWSW3d_1115_'+str(times[i])+'.pkt',losses,'PICKLE')
                else:
                    torch.save(models, '../tmp/maria/3d/lwsw'+str(exp)+'/0/models'+str(splitnum)+'_LWSW3d_1115_'+str(times[i])+'.pt')
                    read_and_proc.save_to_pickle('../tmp/maria/3d/lwsw'+str(exp)+'/0/losses'+str(splitnum)+'_LWSW3d_1115_'+str(times[i])+'.pkt',losses,'PICKLE')

  0%|          | 0/18000 [00:00<?, ?it/s]

(-0.542459870563863, -0.6359118088965873)
(-1.0983300643028762, -1.1105463564373057)
(-1.9837755732965565, -1.8808013045619574)
(-3.092286254614659, -2.847330034649336)
(-4.293773922588387, -4.3158536021352685)


  0%|          | 0/18000 [00:00<?, ?it/s]

(-0.5821636818257716, -0.747037415173495)
(-1.0857503043539776, -1.155065913235442)
(-1.8794919598221271, -1.8369645762839848)
(-3.1229555691439463, -2.9773740048062134)
(-4.302487937317794, -4.321689692889816)


  0%|          | 0/18000 [00:00<?, ?it/s]

...

(-0.6273146921000315, -0.5678012036102142)
(-1.2672406177224607, -1.0458436687270216)
(-2.4004053302142956, -1.3978435225128458)
(-3.98322096594005, -1.5236529023287468)
(-4.244100498022355, -1.6100797660791513)
(-4.281116786415002, -1.6838807157179003)
(-4.296542989826216, -1.7469946260849576)
(-4.306153826236769, -1.8132378680028869)
(-4.312662628008348, -1.8719602433836977)
(-4.3168294825489015, -1.9293094840601777)
(-4.3207104609540625, -1.9887172605640584)
(-4.324399531125305, -2.0412097909682254)
(-4.327305767920542, -2.0969638129534935)
(-4.3307793307852, -2.151339801596751)
(-4.333060974061426, -2.2014432375295967)
(-4.336157675193132, -2.2562834459387484)
(-4.3408535426465455, -2.3070377007109606)
(-4.345501082101282, -2.3580339683688987)
(-4.353320752911614, -2.412936060729876)
(-4.3661981938866905, -2.4637833307220154)
(-4.384610535695075, -2.5204397040881297)
(-4.405148896320805, -2.5769883135382883)
(-4.417440432886206, -2.6275007630991483)
(-4.42406869168124, -2.682997355

  0%|          | 0/18000 [00:00<?, ?it/s]

(-0.627917254892306, -0.5513729084004042)
(-1.26656106272752, -0.9937907206380315)
(-2.4120092762553105, -1.5224092361496762)
(-3.797944713532118, -1.828080174304873)
(-4.224331743802434, -1.9803325720344542)
(-4.260869748962322, -2.0656563842490785)
(-4.282117613445427, -2.1285389686628426)
(-4.299521459504899, -2.1959269841102182)
(-4.317795382479735, -2.2631907465015417)
(-4.340671985592496, -2.336313673872662)
(-4.369764139430314, -2.416140337452739)
(-4.389980478961315, -2.4843732708217527)
(-4.401096800900782, -2.5559637362976093)
(-4.4106744387432615, -2.625431912586648)
(-4.41668775731655, -2.688384601890544)
(-4.4223628285750385, -2.7522491321684663)
(-4.425493344341222, -2.804770371054838)
(-4.428448313423024, -2.8494756039034104)
(-4.431426448059943, -2.8884759379060276)
(-4.433425120362991, -2.9161320661726466)
(-4.434064644552495, -2.9393318630304495)
(-4.435693401918373, -2.9569079612838856)
(-4.437100252146372, -2.9693320493557414)
(-4.43758858853032, -2.980036710981066)

  0%|          | 0/18000 [00:00<?, ?it/s]

IndexError: list index out of range